<a href="https://colab.research.google.com/github/MiguelMendozaG/CANproject/blob/CANmsgs2Dataset/CANmsg_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import argparse
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
model_dir = '/content/gdrive/My Drive/'

Mounted at /content/gdrive/


In [ ]:
car_model = "Colab Notebooks/Car"
#input_file = model_dir + car_model + "/viaje_real_1_MM.log"
input_file = model_dir + car_model + "/viaje_real_1_MM.log"
output_file = model_dir + car_model + "viaje_real_1_MM_5seg"
#CAN_MSG = '0E2,0E6,0EE,116,0FA,0F2'
CAN_MSG = '0E6'
CAN_MSG_txt = CAN_MSG
CAN_MSG = list(CAN_MSG.split(","))
X = []
max_size = 500

In [ ]:
def temporalize(X, y, lookback):
    output_X = []
    output_y = []
    for i in range(len(X)-lookback-1):
        t = []
        for j in range(0,lookback):
            # Gather past records upto the lookback period
            t.append(X[[(i+j)], :])
        output_X.append(t)
        output_y.append(y[i+lookback])
    return output_X, output_y


In [ ]:
def AppendCANMessages(msg):
	X = []
	max_size = 100
	count_msg = 0
	with open(output_file + str(msg) + ".txt", "w") as exit_file:
		with open(input_file, "r") as file:
			lines = file.readlines()
			#if msg in CAN_MSG:
			#print(msg)
			for line in lines:
				line_sep = line.split(" ")
				line_CAN = line_sep[2].split("#")
				#print(line_CAN[0])
				if (line_CAN[0] == msg):
					#print(int((line_CAN[1]),16))
					exit_file.write(str(int((line_CAN[1]),16)) + '\n')
					#exit_file.write(str(line_CAN[1]))
					X.append(int((line_CAN[1]),16))
					if (count_msg < (max_size - 1)):	
						count_msg = count_msg + 1
					else:
						count_msg = 0
						exit_file.write(str('\n'))
			file.close()
			X = np.array([X], np.uint64)
			X = X.transpose()
			timesteps = X.shape[0]
			n_features = X.shape[1]
			#print ("X.shape", X.shape)
			#print ("timesteps", timesteps)
			#print ("n_features ", n_features)
		exit_file.close()
	return X

In [ ]:
X_1 = AppendCANMessages(CAN_MSG[0])
#X_2 = AppendCANMessages(CAN_MSG[1])
#X_3 = AppendCANMessages(CAN_MSG[2])
#X_4 = AppendCANMessages(CAN_MSG[3])
#X_5 = AppendCANMessages(CAN_MSG[4])
#X_6 = AppendCANMessages(CAN_MSG[5])
#X_Complete = np.concatenate((X_1[0:X_5.shape[0]], X_2[0:X_5.shape[0]], X_3[0:X_5.shape[0]],
#                             X_4[0:X_5.shape[0]], X_5, X_6[0:X_5.shape[0]]), axis = 1)

X_Complete = X_1

In [ ]:
print(X_Complete.shape)
print(X_Complete[0:10])

(217600, 1)
[[4765452582132908032]
 [4765452582132908032]
 [4765452582132908032]
 [4720416585859203072]
 [4720416585859203072]
 [4720416585859203072]
 [4666373390330757120]
 [4666373390330757120]
 [4666373390330757120]
 [4666373390330757120]]


In [ ]:
timeseries = X_Complete
timesteps = max_size
X, y = temporalize(X = timeseries, y = np.zeros(len(timeseries)), lookback = timesteps)

In [ ]:
X = np.array(X)
print (X.shape)

In [ ]:
print (X[0,1,0,:])
print (X[1,0,0,:])

In [ ]:
#np.save(model_dir + car_model + '/recorrido_1.npy', X)
np.save(model_dir + car_model + '/viaje_real_1_MM_5seg.npy', X)